In [8]:
using Plots
using Random
using Statistics

function binary_to_decimal(binary, range)
    min_val, max_val = range
    decimal = 0
    for (i, bit) in enumerate(binary)
        decimal += bit * 2^(length(binary) - i)
    end
    
    normalized = decimal / (2^length(binary) - 1)
    return min_val + normalized * (max_val - min_val)
end

function decimal_to_binary(decimal, range, bits_length)
    min_val, max_val = range
    normalized = (decimal - min_val) / (max_val - min_val)
    int_value = round(Int, normalized * (2^bits_length - 1))
    binary = digits(int_value, base=2, pad=bits_length)
    return reverse(binary)
end

function generate_population(size, ranges, bits_per_var)
    population = []
    for _ in 1:size
        x_binary = [rand(0:1) for _ in 1:bits_per_var]
        y_binary = [rand(0:1) for _ in 1:bits_per_var]
        push!(population, (x_binary, y_binary))
    end
    return population
end

function mutate(individual, mutation_rate)
    x_binary, y_binary = individual
    x_mutated = [bit ⊻ (rand() < mutation_rate) for bit in x_binary]
    y_mutated = [bit ⊻ (rand() < mutation_rate) for bit in y_binary]
    
    return (x_mutated, y_mutated)
end

function evaluate_fitness(individual, fitness, ranges, bits_per_var)
    x_binary, y_binary = individual
    
    x = binary_to_decimal(x_binary, ranges[1])
    y = binary_to_decimal(y_binary, ranges[2])
    
    return fitness(x, y), (x, y)
end

function select(population, fitness_values, number_of_parents)
    sorted_idx = sortperm(fitness_values)
    return population[sorted_idx[1:number_of_parents]]
end

function crossover(parents, number_of_children)
    children = []
    for _ in 1:number_of_children
        parent1 = rand(parents)
        parent2 = rand(parents)
        
        x_crosspoint = rand(1:length(parent1[1]))
        y_crosspoint = rand(1:length(parent1[2]))
        
        child_x = vcat(parent1[1][1:x_crosspoint], parent2[1][x_crosspoint+1:end])
        child_y = vcat(parent1[2][1:y_crosspoint], parent2[2][y_crosspoint+1:end])
        
        push!(children, (child_x, child_y))
    end
    return children
end

function genetic_algorithm(fitness, generations, population_size, ranges, mutation_rate, bits_per_var=16)
    Random.seed!(123)
    
    # Генерация начальной популяции
    population = generate_population(population_size, ranges, bits_per_var)
    
    fitness_results = [evaluate_fitness(ind, fitness, ranges, bits_per_var) for ind in population]
    fitness_values = [res[1] for res in fitness_results]
    decoded_values = [res[2] for res in fitness_results]
    
    best_idx = argmin(fitness_values)
    best_solution_binary = population[best_idx]
    best_solution = decoded_values[best_idx]
    best_score = fitness_values[best_idx]
    
    x_range, y_range = ranges
    x_vals = range(x_range[1], x_range[2], length=50)
    y_vals = range(y_range[1], y_range[2], length=50)
    
    anim = Animation()
    anim_contour = Animation()
    
    z_func = [fitness(x, y) for y in y_vals, x in x_vals]
    
    z_min = minimum(z_func) - 5
    z_max = maximum(z_func) + 5
    
    plt = surface(x_vals, y_vals, z_func, 
                 alpha=0.7, 
                 color=:viridis,
                 title="Generation 0",
                 xlabel="X", ylabel="Y", zlabel="f(X,Y)",
                 zlims=(z_min, z_max))
    
    scatter!(plt, [p[1] for p in decoded_values], [p[2] for p in decoded_values], 
            fitness_values, 
            color=:green, markersize=2, label="Популяция")

    scatter!(plt, [best_solution[1]], [best_solution[2]], [best_score], 
            color=:red, markersize=4, marker=:star, 
            label="Лучшая особь ($(round(best_score, digits=4)))")
    
    frame(anim)

    plt_contour = contour(x_vals, y_vals, z_func, 
                         fill=true, 
                         levels=20,
                         color=:viridis,
                         title="Generation 0 (Contour)",
                         xlabel="X", ylabel="Y")
    
    # Добавляем точки популяции на контурный график
    scatter!(plt_contour, [p[1] for p in population], [p[2] for p in population], 
            color=:green, markersize=3, label="Популяция")
    
    # Добавляем лучшее решение на контурный график
    scatter!(plt_contour, [best_solution[1]], [best_solution[2]], 
            color=:red, markersize=5, marker=:star, 
            label="Лучшая особь ($(round(best_score, digits=4)))")
    
    frame(anim_contour)


    
    for generation in 1:generations
        parents = select(population, fitness_values, population_size ÷ 2)
        
        children = crossover(parents, population_size - length(parents))
        
        children = [mutate(child, mutation_rate) for child in children]
        
        population = vcat(parents, children)
        
        fitness_results = [evaluate_fitness(ind, fitness, ranges, bits_per_var) for ind in population]
        fitness_values = [res[1] for res in fitness_results]
        decoded_values = [res[2] for res in fitness_results]
        
        current_best_idx = argmin(fitness_values)
        current_best_binary = population[current_best_idx]
        current_best = decoded_values[current_best_idx]
        current_score = fitness_values[current_best_idx]
        
        if current_score < best_score
            best_solution_binary = current_best_binary
            best_solution = current_best
            best_score = current_score
        end

        if (best_score < 1e-3)
            println("HERE $generation")
            break 
        end
    end

    return best_solution, best_score, best_solution_binary
end


function rosenbrock(x, y)
    return (1 - x)^2 + 100 * (y - x^2)^2
end

# Пример функции Растригина
function rastrigin(x, y)
    return 20 + x^2 + y^2 - 10 * (cos(2π * x) + cos(2π * y))
end

function schwefel(x,y)
    return 418.9829*2 - (x*sin(sqrt(abs(x))) + y*sin(sqrt(abs(y))))
end


fit_function = rosenbrock

params = (
    generations = 50,
    population_size = 200,
    ranges = ((-5.0, 5.0), (-5.0, 5.0)),
    mutation_rate = 0.1,
    bits_per_var = 16
)

t_start = time()

# Запуск алгоритма
@time best_solution, best_score, best_binary = genetic_algorithm(
    fit_function, 
    params.generations, 
    params.population_size, 
    params.ranges, 
    params.mutation_rate,
    params.bits_per_var
)



println("\n\nFinal result:")
println("Best solution: (x, y) = (", round(best_solution[1], digits=4), ", ", 
        round(best_solution[2], digits=4), ")")
println("Function value: f(x, y) = ", round(best_score, digits=4))
println("Binary representation:")
println("X: ", join(best_binary[1]))
println("Y: ", join(best_binary[2]))
elapsed = time() - t_start
println("Классический GA завершен за ", round(elapsed, digits=3), " сек")

  0.724776 seconds (982.67 k allocations: 61.532 MiB, 0.98% gc time, 57.70% compilation time)


Final result:
Best solution: (x, y) = (1.0197, 1.0291)
Function value: f(x, y) = 0.0116
Binary representation:
X: 1001101000011010
Y: 1001101001011000
Классический GA завершен за 0.751 сек
